In [2]:
import pandas as pd
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
from gensim.models import KeyedVectors
import re, pickle

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.covid_hack
tweets_db = db.tweets

In [4]:
def clear_tweet_text(tweet_text):
    tweet_text = re.sub(r'http\S+', '', tweet_text)
    tweet_text = re.sub(r'#\S+', '', tweet_text)
    tweet_text = re.sub(r'@\S+', '', tweet_text)
    tweet_text = re.sub(r'[^\w\s]',' ',tweet_text)
    return tweet_text

def lower_tr(text):
    lowerMap = {ord(u'A'): u'a',ord(u'A'): u'a',ord(u'B'): u'b',ord(u'C'): u'c',ord(u'Ç'): u'ç',ord(u'D'): u'd',ord(u'E'): u'e',ord(u'F'): u'f',ord(u'G'): u'g',ord(u'Ğ'): u'ğ',ord(u'H'): u'h',ord(u'I'): u'ı',ord(u'İ'): u'i',ord(u'J'): u'j',ord(u'K'): u'k',ord(u'L'): u'l',ord(u'M'): u'm',ord(u'N'): u'n',ord(u'O'): u'o',ord(u'Ö'): u'ö',ord(u'P'): u'p',ord(u'R'): u'r',ord(u'S'): u's',ord(u'Ş'): u'ş',ord(u'T'): u't',ord(u'U'): u'u',ord(u'Ü'): u'ü',ord(u'V'): u'v',ord(u'Y'): u'y',ord(u'Z'): u'z',ord(u'W'): u'w',ord(u'Q'): u'q',ord(u'X'): u'x'}
    text = text.translate(lowerMap)
    return text

stop_words = []
with open('stop_words') as f:
    stop_words = f.read().split('\n')

In [11]:
tweet_data_all = []
error_counter = 0

for tweet in tweets_db.find():
    if tweet["full_text"].startswith('RT'):
        continue
    try:
        tweet_data = {}
        tweet_data['id'] = tweet['id']
        tweet_data['created_at'] = tweet['created_at']
        tweet_data['hashtag_id'] = tweet['hashtag_id']
        tweet_data['full_text'] = tweet['full_text']
        tweet_data['retweet_count'] = tweet['retweet_count']
        tweet_data['favorite_count'] = tweet['favorite_count']
        tweet_data['user_follower_count'] = tweet['user']['followers_count']
        tweet_data['lemmatized_text'] = clear_tweet_text(tweet['zemberek_nlp']['lemmatized_text'])
        tweet_data['tokenized_text'] = clear_tweet_text(tweet['zemberek_nlp']['tokenized_text'])
        tweet_data['num_of_hashtags'] = len(tweet['entities']['hashtags'])
        tweet_data['num_of_mentions'] = len(tweet['entities']['user_mentions'])
        tweet_data['num_of_urls'] = len(tweet['entities']['urls'])
        
        processed_text_arr = []
        for word in tweet_data["lemmatized_text"].split():
            if word not in stop_words:
                processed_text_arr.append(word)
        
        processed_text = " ".join(processed_text_arr)
        tweet_data['processed_text'] = processed_text
        
        if len(tweet_data['processed_text'].split()) <= 3:
            continue
        tweet_data_all.append(tweet_data)
    except KeyError as e: 
        print("Error on tweet: {}".format(tweet["full_text"]))
        error_counter += 1

print("Error: {}".format(error_counter))

Error: 0


### Fields To Add DataFrame
* id
* created_at
* hashtag_id
* full_text
* user.followers_count
* zemberek_nlp.lemmatized_text
* zemberek_nlp.tokenized_text

In [12]:
tweet_df = pd.DataFrame(tweet_data_all)

In [13]:
tweet_df.head()

,id,created_at,hashtag_id,full_text,retweet_count,favorite_count,user_follower_count,lemmatized_text,tokenized_text,num_of_hashtags,num_of_mentions,num_of_urls,processed_text
0,1247616672481062913,Tue Apr 07 20:06:08 +0000 2020,#şüpheduymuyorum,@fatihportakal hemen hemen hergün 70’li sayıla...,0,0,276,hemen hemen hergün 70 sayı vermek ölmek için b...,hemen hemen hergün 70 li sayılar veriliyor öl...,1,1,0,hemen hemen hergün 70 sayı vermek ölmek tesadü...
1,1247466322512031744,Tue Apr 07 10:08:42 +0000 2020,#şüpheduymuyorum,@fatihportakal Senin bi amerikan ciciş köpüşü ...,0,1,63,sen bir Amerikan cici kopmak olmak artık hanım,senin bir amerikan cici kopuşu oluşundan artı...,1,2,0,Amerikan cici kopmak hanım
2,1247435972490473472,Tue Apr 07 08:08:06 +0000 2020,#şüpheduymuyorum,#DünyaSağlıkGünü #şüpheduymuyorum #kitlervarla...,1,2,122,tanımak ve tedavi temel belirlemek unsur çok b...,tanı ve tedavide temel belirleyici unsuru ç...,3,0,0,tanımak tedavi temel belirlemek unsur büyük or...
3,1247276012964130816,Mon Apr 06 21:32:29 +0000 2020,#şüpheduymuyorum,@fatihportakal Biri Baban Diğeri Deden Olan Sö...,0,0,527,biri baba diğer de olmak söz dünya dev Amerika...,biri baban diğeri deden olan sözde dünya devl...,1,1,0,baba söz dünya dev Amerika Avrupa dolar euro k...
4,1247182225466146816,Mon Apr 06 15:19:48 +0000 2020,#şüpheduymuyorum,Denklik bekleyen Türk Hekimleri olarak ben ve ...,0,0,18,denklik beklemek Türk hekim olmak ben ve yüz a...,denklik bekleyen türk hekimleri olarak ben ve ...,2,6,0,denklik beklemek Türk hekim arkadaş saha görev...


In [14]:
tweet_df = tweet_df[tweet_df['hashtag_id'] != '#gerçekfikrine']

In [15]:
tweet_df.describe()

,id,retweet_count,favorite_count,user_follower_count,num_of_hashtags,num_of_mentions,num_of_urls
count,4.541600e+04,45416.000000,45416.000000,4.541600e+04,45416.000000,45416.000000,45416.000000
mean,1.247185e+18,2.860996,9.005351,2.164938e+03,1.772745,0.637198,0.050731
std,9.553223e+14,35.398020,151.392778,2.745303e+04,1.704709,1.387295,0.222934
min,1.245388e+18,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1.246115e+18,0.000000,0.000000,9.000000e+00,1.000000,0.000000,0.000000
50%,1.247558e+18,0.000000,1.000000,6.400000e+01,1.000000,0.000000,0.000000
75%,1.247934e+18,0.000000,2.000000,2.980000e+02,2.000000,1.000000,0.000000
max,1.249804e+18,2264.000000,18861.000000,1.410066e+06,29.000000,28.000000,3.000000


In [16]:
tweet_df.to_csv('tweet_data.csv')

In [3]:
tweet_df = pd.read_csv('tweet_data.csv')

In [5]:
word_vectors = KeyedVectors.load_word2vec_format('/home/kemal/Desktop/Workspace/Turkish-Word2Vec/trmodel_me', binary=True)

In [6]:
tweet_vectors_all = []
for ii in range(0, len(tweet_df)):
    hashtag_id = tweet_df.iloc[ii]['hashtag_id']
    if hashtag_id != '#neyinnesi':
        continue
    processed_text = tweet_df.iloc[ii]['processed_text']
    
    tweet_vectors = []
    for word in processed_text.split():
        if word in word_vectors:
            tweet_vectors.append(word_vectors.get_vector(word))
    tweet_vectors_all.append(tweet_vectors)

In [7]:
with open('tweet_vecs', "wb") as f:
    pickle.dump(tweet_vectors_all, f)